In [91]:
# === Librerías principales ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# === Preprocesamiento y modelo ===
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [92]:
import pandas as pd

# Carga individual
df_primera = pd.read_csv('D:/TFG/codigo/datos/indicadores-filtrados-primera-5-cambios.csv', sep=",")
df_premier = pd.read_csv('D:/TFG/codigo/datos/indicadores-filtrados-premier-5-cambios.csv', sep=",")
df_bundesliga = pd.read_csv('D:/TFG/codigo/datos/indicadores-filtrados-bundesliga-5-cambios.csv', sep=",")

# Unir en un solo DataFrame
df = pd.concat([df_primera, df_premier, df_bundesliga], ignore_index=True)

# === Eliminación de columnas irrelevantes o identificadores ===
df.drop(['jornada', 'temporada', 'id_indicadores_equipo_prepartido', 'id_partido'], axis=1, inplace=True)

# === Separación de variables predictoras (X) ===
X = df.drop(['resultado_partido', 'resultado_local', 'resultado_visitante'], axis=1)

# === Variables objetivo ===
y_resultado = df['resultado_partido']           # Clasificación: resultado del partido (1, X, 2)
y_goles_local = df['resultado_local']           # Regresión: goles del equipo local
y_goles_visitante = df['resultado_visitante']   # Regresión: goles del equipo visitante

In [93]:
# === Escalado de los datos ===
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# === División para clasificación del resultado ===
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(
    X_scaled, y_resultado, test_size=0.2, random_state=42
)

# === División para regresión de goles local ===
X_train_loc, X_test_loc, y_train_loc, y_test_loc = train_test_split(
    X_scaled, y_goles_local, test_size=0.2, random_state=42
)

# === División para regresión de goles visitante ===
X_train_vis, X_test_vis, y_train_vis, y_test_vis = train_test_split(
    X_scaled, y_goles_visitante, test_size=0.2, random_state=42
)


### Predicciones resultado

In [94]:
# === Definimos el modelo base ===
knn_model = KNeighborsClassifier()

# === Definimos el grid de hiperparámetros a probar ===
param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

# === Configuramos GridSearchCV ===
grid_search = GridSearchCV(
    estimator=knn_model,
    param_grid=param_grid,
    cv=5,  # validación cruzada de 5 pliegues
    scoring='accuracy',
    n_jobs=-1  # uso de todos los núcleos disponibles
)

# === Entrenamos el modelo con los datos de clasificación ===
grid_search.fit(X_train_cls, y_train_cls)

# Mostramos los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados para clasificación:")
print(grid_search.best_params_)


Mejores hiperparámetros encontrados para clasificación:
{'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'uniform'}


In [95]:
# === Entrenamiento final con los mejores parámetros ===
best_knn_model = grid_search.best_estimator_
best_knn_model.fit(X_train_cls, y_train_cls)

# === Predicción sobre el conjunto de prueba ===
y_pred_cls = best_knn_model.predict(X_test_cls)


In [96]:
# === Evaluación del rendimiento del modelo ===
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Precisión global
accuracy = accuracy_score(y_test_cls, y_pred_cls)
print(f"Accuracy del modelo KNN en el conjunto de prueba: {accuracy:.4f}")

# Matriz de confusión
print("Matriz de confusión:")
print(confusion_matrix(y_test_cls, y_pred_cls))

Accuracy del modelo KNN en el conjunto de prueba: 0.4492
Matriz de confusión:
[[117  28  25]
 [ 58  27  13]
 [ 59  12  15]]


In [97]:
from sklearn.inspection import permutation_importance

# Obtener los nombres de las variables predictoras
feature_names = X.columns

# Para KNN, la importancia de variables no es directa, pero podemos usar Permutation Importance

# Calculamos la importancia por permutación sobre el conjunto de test
result = permutation_importance(best_knn_model, X_test_cls, y_test_cls, n_repeats=1, random_state=42, n_jobs=-1)

# Obtenemos las 10 variables más importantes y sus importancias
importances = result.importances_mean
indices = np.argsort(importances)[::-1][:10]

print("Top 10 variables más importantes y sus pesos:")
for idx in indices:
    print(f"{feature_names[idx]}: {importances[idx]:.4f}")

# Filtramos las variables relacionadas con sustituciones
sustitucion_mask = feature_names.str.contains('cambio', case=False)
all_mask = feature_names.str.contains('e', case=False)
sustitucion_importance = abs(importances[sustitucion_mask]).sum()
all_importance = abs(importances[all_mask]).sum()
print()

print(f"\Porcentaje de importancia de variables relacionadas con sustituciones: {sustitucion_importance/all_importance:.4f}")

Top 10 variables más importantes y sus pesos:
proporcion visitante cambios antes descanso en general: 0.0311
proporcion local cambios delanteros a centrocampistas en general: 0.0311
proporcion local cambios alineacion defensa sitio: 0.0254
proporcion visitante cambios lesionados en sitio: 0.0254
proporcion visitante corners en contra en general: 0.0254
proporcion visitante corners a favor en general: 0.0254
proporcion local cambios en general: 0.0226
proporcion local cambios centrocampistas a delanteros en general: 0.0226
proporcion local cambios alineacion delantero en general: 0.0226
proporcion local corners a favor en sitio: 0.0226

\Porcentaje de importancia de variables relacionadas con sustituciones: 0.4913


### Predicción equipo local

In [98]:
from sklearn.neighbors import KNeighborsRegressor

# === Modelo base ===
knn_reg = KNeighborsRegressor()

# === Grid de hiperparámetros ===
param_grid_reg = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

# === Búsqueda de los mejores hiperparámetros ===
grid_search_reg = GridSearchCV(
    estimator=knn_reg,
    param_grid=param_grid_reg,
    cv=5,
    scoring='neg_root_mean_squared_error',  # Minimizar RMSE
    n_jobs=-1
)

# Ajuste con los datos de goles locales
grid_search_reg.fit(X_train_loc, y_train_loc)

# Mostrar los mejores parámetros encontrados
print("Mejores hiperparámetros para la regresión de goles local:")
print(grid_search_reg.best_params_)


Mejores hiperparámetros para la regresión de goles local:
{'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'distance'}


In [99]:
# === Entrenamiento final con los mejores parámetros ===
best_knn_reg = grid_search_reg.best_estimator_
best_knn_reg.fit(X_train_loc, y_train_loc)

# === Predicción sobre el conjunto de prueba ===
y_pred_loc = best_knn_reg.predict(X_test_loc)


In [100]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# === Cálculo del error cuadrático medio ===
rmse_loc = np.sqrt(mean_squared_error(y_test_loc, y_pred_loc))
r2_loc = r2_score(y_test_loc, y_pred_loc)

# === Resultados ===
print(f"RMSE (goles local): {rmse_loc:.4f}")
print(f"R² Score (goles local): {r2_loc:.4f}")

RMSE (goles local): 1.3116
R² Score (goles local): -0.0705


In [101]:
from sklearn.inspection import permutation_importance

# Obtener los nombres de las variables predictoras
feature_names = X.columns

# Para KNN, la importancia de variables no es directa, pero podemos usar Permutation Importance

# Calculamos la importancia por permutación sobre el conjunto de test
result = permutation_importance(best_knn_reg, X_test_loc, y_test_loc, n_repeats=1, random_state=42, n_jobs=-1)

# Obtenemos las 10 variables más importantes y sus importancias
importances = result.importances_mean
indices = np.argsort(importances)[::-1][:10]

print("Top 10 variables más importantes y sus pesos:")
for idx in indices:
    print(f"{feature_names[idx]}: {importances[idx]:.4f}")

# Filtramos las variables relacionadas con sustituciones
sustitucion_mask = feature_names.str.contains('cambio', case=False)
all_mask = feature_names.str.contains('e', case=False)
sustitucion_importance = abs(importances[sustitucion_mask]).sum()
all_importance = abs(importances[all_mask]).sum()
print()
print(f"\Porcentaje de importancia de variables relacionadas con sustituciones: {sustitucion_importance/all_importance:.4f}")

Top 10 variables más importantes y sus pesos:
proporcion visitante cambios defensas a centrocampistas en sitio: 0.0157
proporcion local cambios delanteros a centrocampistas sitio: 0.0129
porcentaje local mas 0,5 marcados en general: 0.0112
proporcion visitante cambios alineacion delantero en general: 0.0103
proporcion local cambios en general: 0.0097
proporcion local cambios delanteros a centrocampistas en general: 0.0087
proporcion local cambios 45 a 60 en general: 0.0087
proporcion visitante cambios defensas a centrocampistas en general: 0.0083
porcentaje visitante mas 1,5 en sitio: 0.0080
porcentaje local perdidos en general: 0.0077

\Porcentaje de importancia de variables relacionadas con sustituciones: 0.4718


### Predicción equipo visitante

In [102]:
# === Modelo base para regresión de goles visitante ===
knn_reg_vis = KNeighborsRegressor()

# === Grid de hiperparámetros ===
param_grid_reg_vis = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

# === Búsqueda de hiperparámetros con validación cruzada ===
grid_search_reg_vis = GridSearchCV(
    estimator=knn_reg_vis,
    param_grid=param_grid_reg_vis,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1
)

# Ajustar el modelo
grid_search_reg_vis.fit(X_train_vis, y_train_vis)

# Mostrar mejores hiperparámetros
print("Mejores hiperparámetros para la regresión de goles visitante:")
print(grid_search_reg_vis.best_params_)


Mejores hiperparámetros para la regresión de goles visitante:
{'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'uniform'}


In [103]:
# === Entrenamiento final con mejores parámetros ===
best_knn_reg_vis = grid_search_reg_vis.best_estimator_
best_knn_reg_vis.fit(X_train_vis, y_train_vis)

# === Predicción sobre el conjunto de prueba ===
y_pred_vis = best_knn_reg_vis.predict(X_test_vis)


In [104]:
# === Evaluación estándar ===
rmse_vis = np.sqrt(mean_squared_error(y_test_vis, y_pred_vis))
r2_vis = r2_score(y_test_vis, y_pred_vis)

print(f"RMSE (goles visitante): {rmse_vis:.4f}")
print(f"R² Score (goles visitante): {r2_vis:.4f}")

RMSE (goles visitante): 1.0862
R² Score (goles visitante): -0.0365


In [105]:
from sklearn.inspection import permutation_importance

# Obtener los nombres de las variables predictoras
feature_names = X.columns

# Para KNN, la importancia de variables no es directa, pero podemos usar Permutation Importance

# Calculamos la importancia por permutación sobre el conjunto de test
result = permutation_importance(best_knn_reg_vis, X_test_vis, y_test_vis, n_repeats=1, random_state=42, n_jobs=-1)

# Obtenemos las 10 variables más importantes y sus importancias
importances = result.importances_mean
indices = np.argsort(importances)[::-1][:10]

print("Top 10 variables más importantes y sus pesos:")
for idx in indices:
    print(f"{feature_names[idx]}: {importances[idx]:.4f}")

# Filtramos las variables relacionadas con sustituciones
sustitucion_mask = feature_names.str.contains('cambio', case=False)
all_mask = feature_names.str.contains('e', case=False)
sustitucion_importance = abs(importances[sustitucion_mask]).sum()
all_importance = abs(importances[all_mask]).sum()
print()

print(f"\Porcentaje de importancia de variables relacionadas con sustituciones: {sustitucion_importance/all_importance:.4f}")

Top 10 variables más importantes y sus pesos:
proporcion visitante cambios 76 a final en sitio: 0.0119
porcentaje visitante perdidos en general: 0.0108
proporcion local rojas en sitio: 0.0103
proporcion local cambios antes descanso en general: 0.0083
proporcion local cambios centrocampistas a delanteros en general: 0.0081
porcentaje local mas 2,5 en sitio: 0.0073
proporcion local cambios 61 a 75 sitio: 0.0064
porcentaje local mas 0,5 encajados en sitio: 0.0061
proporcion local cambios alineacion defensa sitio: 0.0059
proporcion local cambios alineacion delantero sitio: 0.0059

\Porcentaje de importancia de variables relacionadas con sustituciones: 0.4638
